In [31]:
from pathlib import Path

import pandas as pd
import re


from sqlalchemy.testing.plugin.plugin_base import logging

DATA_PATH = Path("/home/wby/projects/week5/data/recipes.csv")
BATCH_SIZE = 1000
KEY_FIELD = [
        "RecipeId",                  # 食谱唯一ID（关联片段与原数据，必选）
        "Name",                      # 食谱名称（生成回答时展示，必选）
        "RecipeIngredientQuantities",# 食材用量（如“500g”，核心检索字段）
        "RecipeIngredientParts",     # 食材名称（如“鸡肉”，核心检索字段）
        "RecipeCategory",            # 食物类别
        "RecipeInstructions",        # 烹饪步骤（生成回答的核心内容，必选）
        "TotalTime",                 # 总耗时（用户可能筛选“30分钟内”，可选）
        "Calories",                  # 热量（用户可能筛选“低热量”，可选）
        "RecipeServings"             # 份数（后续Agent单位转换用，可选）
    ]

In [32]:

def load_data():
    if not DATA_PATH.exists():
        error_msg = f"文件不存在"
        logging.error
        raise FileNotFoundError(error_msg)

    try:
        batch_generator = pd.read_csv(
            DATA_PATH,
            chunksize=BATCH_SIZE,
            usecols=KEY_FIELD,
            encoding="utf-8",
        )
        return batch_generator
    except Exception as e:
        raise



In [33]:
data_list = []
def test_loader():
    try:
        batch_generator = load_data()
        batch_num = 0
        for batch in batch_generator:
            print(f"第{batch_num+1}组数据")
            print(batch.info())
            batch_num += 1
            data_list.append(batch)
            if batch_num >2:
                break
    except Exception as e:
        raise

test_loader()

df = pd.concat(data_list,ignore_index=True)

第1组数据
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    1000 non-null   int64  
 1   Name                        1000 non-null   object 
 2   TotalTime                   1000 non-null   object 
 3   RecipeCategory              1000 non-null   object 
 4   RecipeIngredientQuantities  999 non-null    object 
 5   RecipeIngredientParts       1000 non-null   object 
 6   Calories                    1000 non-null   float64
 7   RecipeServings              688 non-null    float64
 8   RecipeInstructions          1000 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 70.4+ KB
None
第2组数据
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 1000 to 1999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------           

In [34]:
null_count = df.isnull().sum()
null_avage = (null_count / len(df)).round(3)
missing_df = pd.DataFrame({
    'missing_number':null_count,
    'missing_average':null_avage,
}).sort_values(by=['missing_number'], ascending=False)
print(missing_df)


                            missing_number  missing_average
RecipeServings                        1075            0.358
RecipeCategory                           1            0.000
RecipeIngredientQuantities               1            0.000
RecipeId                                 0            0.000
Name                                     0            0.000
TotalTime                                0            0.000
RecipeIngredientParts                    0            0.000
Calories                                 0            0.000
RecipeInstructions                       0            0.000


In [5]:
# 用字符串长度初步分类（不同格式长度可能不同）
empyt_str = df[df['TotalTime'].str.len() == 0]
print(empyt_str)
# 统计不同格式的数量（比如“30 min”“1 h 20 min”“PT30M”等）
print("\n=== TotalTime格式分布 ===")
# 用字符串长度初步分类（不同格式长度可能不同）
time_length_count = df["TotalTime"].str.len().value_counts().sort_values()
print(time_length_count)

#没有M,h,H,min的东西
print("\n=== 查看非M,h,H,min的字段 ===")
on_time_values = df[~df['TotalTime'].str.contains('min|h|H|M',na=False)][['RecipeId','TotalTime','RecipeServings']]
print(on_time_values)

#查看卡路里异常值
print('\n==== 查看卡路里异常值 ===')
on_corli = df[df['Calories'] <= 0][['RecipeId','Name','Calories']]
print(on_corli)



Empty DataFrame
Columns: [RecipeId, Name, TotalTime, RecipeCategory, RecipeIngredientQuantities, RecipeIngredientParts, Calories, RecipeServings, RecipeInstructions]
Index: []

=== TotalTime格式分布 ===
TotalTime
9       3
8      25
6      92
7     526
5     973
4    1381
Name: count, dtype: int64

=== 查看非M,h,H,min的字段 ===
      RecipeId TotalTime  RecipeServings
321        387      PT0S             1.0
610        707      PT0S             NaN
613        715      PT0S             1.0
616        721      PT0S             4.0
618        736      PT0S            12.0
...        ...       ...             ...
2988      4647      PT0S            20.0
2989      4648      PT0S             8.0
2991      4650      PT0S             1.0
2995      4657      PT0S             1.0
2999      4661      PT0S             8.0

[1139 rows x 3 columns]

==== 查看卡路里异常值 ===
      RecipeId                                       Name  Calories
342        410                                 Cafe Latte       0.0
433     

In [6]:
def count_num(x):
    if pd.isna(x):
        return 0
    return len([i.strip() for i in x.split(",") if i.strip()])

df['Quanti_num'] = df['RecipeIngredientQuantities'].apply(count_num)
df['Ingredient_num'] = df['RecipeIngredientParts'].apply(count_num)
value_not_equier = df[df['Quanti_num'] != df['Ingredient_num']][
        ["RecipeId","RecipeIngredientParts","RecipeIngredientQuantities","Quanti_num", "Ingredient_num"]
    ]
print("针对食材数量和食材不匹配得到的数据形状")
print(value_not_equier.shape)

print("针对食材数量和食材不匹配的具体情况")
print(value_not_equier.info())

针对食材数量和食材不匹配得到的数据形状
(2231, 5)
针对食材数量和食材不匹配的具体情况
<class 'pandas.core.frame.DataFrame'>
Index: 2231 entries, 1 to 2999
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   RecipeId                    2231 non-null   int64 
 1   RecipeIngredientParts       2231 non-null   object
 2   RecipeIngredientQuantities  2230 non-null   object
 3   Quanti_num                  2231 non-null   int64 
 4   Ingredient_num              2231 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 104.6+ KB
None


In [7]:

# 查看前3条步骤的完整内容
for i in range(20):
    print(f"第{i+1}条食谱")
    print(f"RecipeId {value_not_equier.iloc[i]['RecipeId']} : {value_not_equier.iloc[i]['RecipeIngredientQuantities']}")
    print(f"RecipeId {value_not_equier.iloc[i]['RecipeId']} : {value_not_equier.iloc[i]['RecipeIngredientParts']}")

# 检查步骤是否为空字符串（即使isnull显示无缺失，可能有“空字符串”）
empty_instructions = len(df[df["RecipeInstructions"].str.strip() == ""])
print(f"\n步骤为空字符串的样本数量：{empty_instructions}")


第1条食谱
RecipeId 39 : c("1", "4", "2", "2", "8", "1/4", "8", "1/2", "1", "1", "1/4", "1/4", "1/2", "1/4", "2", "3", NA, "2", "1", "1", "8", "2", "1/3", "1/3", "1/3", "6")
RecipeId 39 : c("saffron", "milk", "hot green chili peppers", "onions", "garlic", "clove", "peppercorns", "cardamom seed", "cumin seed", "poppy seed", "mace", "cilantro", "mint leaf", "fresh lemon juice", "plain yogurt", "boneless chicken", "salt", "ghee", "onion", "tomatoes", "basmati rice", "long-grain rice", "raisins", "cashews", "eggs")
第2条食谱
RecipeId 40 : c("1 1/2", "1", NA, "1 1/2", NA, "3/4")
RecipeId 40 : c("sugar", "lemons, rind of", "lemon, zest of", "fresh water", "fresh lemon juice")
第3条食谱
RecipeId 41 : c("12", "1", "2", "1", "10", "1", "3", "2", "2", "2", "1", "2", "1/2", "1/4", "4")
RecipeId 41 : c("extra firm tofu", "eggplant", "zucchini", "mushrooms", "soy sauce", "low sodium soy sauce", "olive oil", "maple syrup", "honey", "red wine vinegar", "lemon juice", "garlic cloves", "mustard powder", "black pepp

In [8]:

#进行格式统一
def iso_to_minutes(iso_str):
    # 处理异常值：PT0S直接返回None（后续统一填充）
    if iso_str == "PT0S":
        return None

    # 用正则表达式提取“数字+单位”（支持整数/小数，如PT1.5H、PT20M）
    # 匹配规则：匹配1个或多个数字（可含小数点）+ 单位（D/H/M/S）
    pattern = r"(\d+\.?\d*)([DHMS])"
    matches = re.findall(pattern, iso_str)

    total_num = 0.0
    for number,unit in matches:
        num = float(number)
        if unit == 'D':
            total_num += num * 1440
        elif unit == 'H':
            total_num += num * 60
        elif unit == 'M':
            total_num += num
        elif unit == 'S':
            total_num += num / 60
    return round(total_num)

test_cases = ["PT30M", "PT1H15M", "PT2D3H", "PT45S", "PT0S", "PT1.5H"]
for t in test_cases:
    result = iso_to_minutes(t)
    print(f"转为分钟{result}")

转为分钟30
转为分钟75
转为分钟3060
转为分钟1
转为分钟None
转为分钟90


In [9]:
df['TotalTime'] =df["TotalTime"].apply(iso_to_minutes)

In [10]:
valid_time = df['TotalTime'].notnull().sum() / len(df)
unvalid_time = df['TotalTime'].isnull().sum() / len(df)
print(valid_time)
print(unvalid_time)


group_means = df.groupby('RecipeCategory')['TotalTime'].mean()


for category,mean_time in group_means.items():
    print(f"食谱类别：{category} → 平均时间：{mean_time:.1f} 分钟")  # .1f 保留1位小数

0.6203333333333333
0.37966666666666665
食谱类别：< 15 Mins → 平均时间：12.1 分钟
食谱类别：< 30 Mins → 平均时间：23.0 分钟
食谱类别：< 4 Hours → 平均时间：77.5 分钟
食谱类别：< 60 Mins → 平均时间：46.5 分钟
食谱类别：Apple → 平均时间：36.0 分钟
食谱类别：Asian → 平均时间：44.3 分钟
食谱类别：Bar Cookie → 平均时间：87.2 分钟
食谱类别：Bass → 平均时间：55.0 分钟
食谱类别：Beans → 平均时间：51.5 分钟
食谱类别：Beef Organ Meats → 平均时间：75.5 分钟
食谱类别：Berries → 平均时间：55.0 分钟
食谱类别：Beverages → 平均时间：4756.7 分钟
食谱类别：Black Beans → 平均时间：95.0 分钟
食谱类别：Brazilian → 平均时间：217.0 分钟
食谱类别：Breads → 平均时间：59.5 分钟
食谱类别：Breakfast → 平均时间：672.4 分钟
食谱类别：Broil/Grill → 平均时间：25.0 分钟
食谱类别：Brown Rice → 平均时间：150.0 分钟
食谱类别：Cajun → 平均时间：30.0 分钟
食谱类别：Canadian → 平均时间：nan 分钟
食谱类别：Candy → 平均时间：84.9 分钟
食谱类别：Caribbean → 平均时间：10.0 分钟
食谱类别：Cauliflower → 平均时间：43.2 分钟
食谱类别：Cheese → 平均时间：75.5 分钟
食谱类别：Cheesecake → 平均时间：201.7 分钟
食谱类别：Chicken → 平均时间：62.4 分钟
食谱类别：Chicken Breast → 平均时间：144.0 分钟
食谱类别：Chicken Livers → 平均时间：212.2 分钟
食谱类别：Chicken Thigh & Leg → 平均时间：68.6 分钟
食谱类别：Chinese → 平均时间：10.0 分钟
食谱类别：Chowders → 平均时间：110.4 分钟
食谱类别：Christmas → 平均时间：125.

In [11]:
for category,mean_time in group_means.items():
    if pd.isna(mean_time):
        print(category)


Canadian
Elk
Japanese
Kid Friendly
Lebanese
Medium Grain Rice
Native American
Papaya
Penne
Potluck
Rabbit
Strawberry
Swiss
Thai
Vegan
Vietnamese
Whole Turkey
Yam/Sweet Potato


In [12]:
null_cat_time_map = {
    # 第一类：食材/菜品类型
    "Papaya": 15,
    "Strawberry": 15,
    "Penne": 25,
    "Yam/Sweet Potato": 35,
    "Whole Turkey": 180,
    "Rabbit": 90,
    # 第二类：地域菜系
    "Japanese": 30,
    "Thai": 35,
    "Vietnamese": 30,
    "Lebanese": 45,
    "Swiss": 50,
    "Canadian": 45,
    # 第三类：饮食场景/需求
    "Kid Friendly": 20,
    "Potluck": 60,
    "Vegan": 35,
    "Native American": 40,
    "Medium Grain Rice": 30,
    "Elk": 90
}

#优先使用人工映射表来进行数据的填充
for cat,time in null_cat_time_map.items():
    mask = (df['RecipeCategory'] == cat)&(df['TotalTime'].isna())
    df.loc[mask, 'TotalTime'] = time

#之后使用平均值来进行填充
group_means = df.groupby('RecipeCategory')['TotalTime'].mean()
for cat,time in group_means.items():
    if pd.isna(time):
        continue
    mask = (df['RecipeCategory'] == cat)&(df['TotalTime'].isna())
    df.loc[mask, 'TotalTime'] = time

#如果都没有，那么用中位数进行填充
gloabal_median = df[df['TotalTime'].notna()]['TotalTime'].median()
mask = df['TotalTime'].isna()
df.loc[mask, 'TotalTime'] = gloabal_median


print('尝试打印空值')
for cat,time in group_means.items():
    print(f"食谱类别：{cat} → 平均时间：{time:.1f} 分钟")  # .1f 保留1位小数


尝试打印空值
食谱类别：< 15 Mins → 平均时间：12.1 分钟
食谱类别：< 30 Mins → 平均时间：23.0 分钟
食谱类别：< 4 Hours → 平均时间：77.5 分钟
食谱类别：< 60 Mins → 平均时间：46.5 分钟
食谱类别：Apple → 平均时间：36.0 分钟
食谱类别：Asian → 平均时间：44.3 分钟
食谱类别：Bar Cookie → 平均时间：87.2 分钟
食谱类别：Bass → 平均时间：55.0 分钟
食谱类别：Beans → 平均时间：51.5 分钟
食谱类别：Beef Organ Meats → 平均时间：75.5 分钟
食谱类别：Berries → 平均时间：55.0 分钟
食谱类别：Beverages → 平均时间：4756.7 分钟
食谱类别：Black Beans → 平均时间：95.0 分钟
食谱类别：Brazilian → 平均时间：217.0 分钟
食谱类别：Breads → 平均时间：59.5 分钟
食谱类别：Breakfast → 平均时间：672.4 分钟
食谱类别：Broil/Grill → 平均时间：25.0 分钟
食谱类别：Brown Rice → 平均时间：150.0 分钟
食谱类别：Cajun → 平均时间：30.0 分钟
食谱类别：Canadian → 平均时间：45.0 分钟
食谱类别：Candy → 平均时间：84.9 分钟
食谱类别：Caribbean → 平均时间：10.0 分钟
食谱类别：Cauliflower → 平均时间：43.2 分钟
食谱类别：Cheese → 平均时间：75.5 分钟
食谱类别：Cheesecake → 平均时间：201.7 分钟
食谱类别：Chicken → 平均时间：62.4 分钟
食谱类别：Chicken Breast → 平均时间：144.0 分钟
食谱类别：Chicken Livers → 平均时间：212.2 分钟
食谱类别：Chicken Thigh & Leg → 平均时间：68.6 分钟
食谱类别：Chinese → 平均时间：10.0 分钟
食谱类别：Chowders → 平均时间：110.4 分钟
食谱类别：Christmas → 平均时间：125.0 分钟
食谱类别：Chutneys → 平均时间：87.5 

In [13]:
for cat,time in group_means.items():
    print(f"食谱类别：{cat} → 平均时间：{time:.1f} 分钟")  # .1f 保留1位小数

食谱类别：< 15 Mins → 平均时间：12.1 分钟
食谱类别：< 30 Mins → 平均时间：23.0 分钟
食谱类别：< 4 Hours → 平均时间：77.5 分钟
食谱类别：< 60 Mins → 平均时间：46.5 分钟
食谱类别：Apple → 平均时间：36.0 分钟
食谱类别：Asian → 平均时间：44.3 分钟
食谱类别：Bar Cookie → 平均时间：87.2 分钟
食谱类别：Bass → 平均时间：55.0 分钟
食谱类别：Beans → 平均时间：51.5 分钟
食谱类别：Beef Organ Meats → 平均时间：75.5 分钟
食谱类别：Berries → 平均时间：55.0 分钟
食谱类别：Beverages → 平均时间：4756.7 分钟
食谱类别：Black Beans → 平均时间：95.0 分钟
食谱类别：Brazilian → 平均时间：217.0 分钟
食谱类别：Breads → 平均时间：59.5 分钟
食谱类别：Breakfast → 平均时间：672.4 分钟
食谱类别：Broil/Grill → 平均时间：25.0 分钟
食谱类别：Brown Rice → 平均时间：150.0 分钟
食谱类别：Cajun → 平均时间：30.0 分钟
食谱类别：Canadian → 平均时间：45.0 分钟
食谱类别：Candy → 平均时间：84.9 分钟
食谱类别：Caribbean → 平均时间：10.0 分钟
食谱类别：Cauliflower → 平均时间：43.2 分钟
食谱类别：Cheese → 平均时间：75.5 分钟
食谱类别：Cheesecake → 平均时间：201.7 分钟
食谱类别：Chicken → 平均时间：62.4 分钟
食谱类别：Chicken Breast → 平均时间：144.0 分钟
食谱类别：Chicken Livers → 平均时间：212.2 分钟
食谱类别：Chicken Thigh & Leg → 平均时间：68.6 分钟
食谱类别：Chinese → 平均时间：10.0 分钟
食谱类别：Chowders → 平均时间：110.4 分钟
食谱类别：Christmas → 平均时间：125.0 分钟
食谱类别：Chutneys → 平均时间：87.5 分钟
食谱类别

In [14]:
print(df.info())
df.dropna(subset=['RecipeCategory'], inplace=True)
df.dropna(subset=['RecipeIngredientQuantities'], inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    3000 non-null   int64  
 1   Name                        3000 non-null   object 
 2   TotalTime                   3000 non-null   float64
 3   RecipeCategory              2999 non-null   object 
 4   RecipeIngredientQuantities  2999 non-null   object 
 5   RecipeIngredientParts       3000 non-null   object 
 6   Calories                    3000 non-null   float64
 7   RecipeServings              1925 non-null   float64
 8   RecipeInstructions          3000 non-null   object 
 9   Quanti_num                  3000 non-null   int64  
 10  Ingredient_num              3000 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 257.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 2998 entries, 0 to 2999
Data columns (

In [15]:
group_means_num = df.groupby('RecipeCategory')['RecipeServings'].mean().round()
for cat,num in group_means_num.items():
    print(f"食谱类别{cat}->{num}")

print("尝试打印空值类别")
for cat,num in group_means_num.items():
    if pd.isna(num):
        print(cat)

食谱类别< 15 Mins->6.0
食谱类别< 30 Mins->5.0
食谱类别< 4 Hours->4.0
食谱类别< 60 Mins->8.0
食谱类别Apple->5.0
食谱类别Asian->2.0
食谱类别Bar Cookie->22.0
食谱类别Bass->3.0
食谱类别Beans->7.0
食谱类别Beef Organ Meats->7.0
食谱类别Berries->nan
食谱类别Beverages->7.0
食谱类别Black Beans->7.0
食谱类别Brazilian->8.0
食谱类别Breads->10.0
食谱类别Breakfast->6.0
食谱类别Broil/Grill->1.0
食谱类别Brown Rice->7.0
食谱类别Cajun->8.0
食谱类别Canadian->6.0
食谱类别Candy->14.0
食谱类别Caribbean->10.0
食谱类别Cauliflower->4.0
食谱类别Cheese->5.0
食谱类别Cheesecake->10.0
食谱类别Chicken->5.0
食谱类别Chicken Breast->5.0
食谱类别Chicken Livers->4.0
食谱类别Chicken Thigh & Leg->5.0
食谱类别Chinese->nan
食谱类别Chowders->5.0
食谱类别Christmas->1.0
食谱类别Chutneys->4.0
食谱类别Citrus->nan
食谱类别Clear Soup->5.0
食谱类别Coconut->4.0
食谱类别Colombian->4.0
食谱类别Corn->5.0
食谱类别Crab->9.0
食谱类别Crawfish->4.0
食谱类别Curries->4.0
食谱类别Dessert->10.0
食谱类别Drop Cookies->34.0
食谱类别Dutch->6.0
食谱类别Easy->nan
食谱类别Elk->4.0
食谱类别European->7.0
食谱类别For Large Groups->50.0
食谱类别Free Of...->4.0
食谱类别Frozen Desserts->6.0
食谱类别Fruit->6.0
食谱类别Gelatin->10.0
食谱类别German->3.0
食谱类别Grains->nan

In [16]:
serving_map = {
    # 第一类：食材量/菜品规模
    "Whole Turkey": 10,
    "Berries": 4,
    "Citrus": 4,
    "Strawberry": 4,
    "Papaya": 4,
    "Perch": 2,
    # 第二类：饮食场景/需求
    "Kid Friendly": 2,
    "Easy": 2,
    "No Cook": 2,
    "Lactose Free": 4,
    "Spicy": 4,
    # 第三类：地域/食材类型
    "Chinese": 4,
    "Hungarian": 4,
    "Southwest Asia (middle East)": 4,
    "Grains": 4
}

#优先考虑人工表
for cat,num in serving_map.items():
    mask = (df['RecipeCategory'] == cat)&(df['RecipeServings'].isna())
    df.loc[mask, 'RecipeServings'] = num

# 再考虑使用平均值
group_means_num = df.groupby('RecipeCategory')['RecipeServings'].mean().round()
for cat,num in group_means_num.items():
    if pd.isna(num):
        continue
    mask = (df['RecipeCategory'] == cat)&(df['RecipeServings'].isna())
    df.loc[mask, 'RecipeServings'] = num

# 最后考虑总的中位数
gloabal_median_num = df[df['RecipeServings'].notna()]['RecipeServings'].median().round()
mask = df['RecipeServings'].isna()
df.loc[mask, 'RecipeServings'] = gloabal_median_num

In [17]:
def fillByGlobalNum(x):
    if pd.isna(x):
        return gloabal_median_num
    return x

df['RecipeServings'] = df['RecipeServings'].apply(fillByGlobalNum)
print(df['RecipeServings'].isna().sum())

0


In [18]:
print("查看卡路里异常")
calro = df[df['Calories'] <= 0][['RecipeCategory','Name','Calories']]
print(calro)

查看卡路里异常
       RecipeCategory                                       Name  Calories
342         Beverages                                 Cafe Latte       0.0
433    Very Low Carbs                             Habanero Honey       0.0
434           Gelatin                             Glowing Jell-O       0.0
458    Chicken Breast           Gray Girls' Easy Chicken Fingers       0.0
466           Peppers                       Grilled Bell Peppers       0.0
613           Chicken                            Unusual Chicken       0.0
656         Breakfast                         Homemade Pop-Tarts       0.0
788         Vegetable                                Parsnip Pie       0.0
938          European                                    Caprese       0.0
965            Cheese                    Homemade Cottage Cheese       0.0
968          European                                 Bruschetta       0.0
1029          Dessert                               Cookie Icing       0.0
1115     Lunch/Sn

In [19]:
calorie_map = {
    # 饮品/酱料类（低卡）
    "Beverages": 80,          # 拿铁咖啡约80大卡
    "Herbal Vinegar": 5,      # 草本醋几乎无热量，取5大卡
    "Gelatin": 70,            # 果冻约70大卡
    "Cookie Icing": 120,      # 糖霜约120大卡/份
    "Lemon Marmalade": 100,   # 柠檬酱约100大卡

    # 蔬菜/水果类（中低卡）
    "Vegetable": 60,          # 蔬菜派约60大卡
    "Peppers": 30,            # 烤甜椒约30大卡
    "Fruit": 50,              # 水果类约50大卡
    "Native American": 80,    # 南瓜约80大卡
    "Vegan": 90,              # 纯素南瓜泥约90大卡

    # 主食/肉类/甜点（中高卡）
    "Chicken Breast": 200,    # 炸鸡条约200大卡
    "Chicken": 220,           # 鸡肉料理约220大卡
    "Meat": 300,              # 烤肉约300大卡
    "Cheese": 180,            # 奶酪约180大卡
    "European": 250,          # 意大利面/烤面包约250大卡
    "Dessert": 350,           # 自制糕点约350大卡
    "Lunch/Snacks": 200,      # 三明治/芝士条约200大卡
    "Very Low Carbs": 150     # 低卡蜂蜜约150大卡（含少量糖）
}

#优先用表
for cat,cal in calorie_map.items():
    mask = (df['RecipeCategory'] == cat)&(df['Calories']<=0)
    df.loc[mask, 'Calories'] = cal

#再考虑用平均值
group_means_cal = df.groupby('RecipeCategory')['Calories'].mean()
print(group_means_cal[group_means_cal<=0])
for cat,cal in group_means_cal.items():
    if cal <= 0:
        continue
    mask = (df['RecipeCategory'] == cat)&(df['Calories']<=0)
    df.loc[mask, 'Calories'] = cal

#最后考虑用中位数
gloabal_median_cal = df['Calories'].median()
mask = df['Calories']<=0
df.loc[mask, 'Calories'] = gloabal_median_cal

print(df[df['Calories']<=0])

Series([], Name: Calories, dtype: float64)
Empty DataFrame
Columns: [RecipeId, Name, TotalTime, RecipeCategory, RecipeIngredientQuantities, RecipeIngredientParts, Calories, RecipeServings, RecipeInstructions, Quanti_num, Ingredient_num]
Index: []


In [20]:
def count_num(x):
    if pd.isna(x):
        return 0
    return len([i.strip() for i in x.split(",") if i.strip()])

df['Quanti_num'] = df['RecipeIngredientQuantities'].apply(count_num)
df['Ingredient_num'] = df['RecipeIngredientParts'].apply(count_num)
value_not_equier = df[df['Quanti_num'] != df['Ingredient_num']][
        ["RecipeId","RecipeIngredientParts","RecipeIngredientQuantities","Quanti_num", "Ingredient_num"]
    ]
print("针对食材数量和食材不匹配得到的数据形状")
print(value_not_equier.shape)

print("针对食材数量和食材不匹配的具体情况")
print(value_not_equier)

针对食材数量和食材不匹配得到的数据形状
(2229, 5)
针对食材数量和食材不匹配的具体情况
      RecipeId                              RecipeIngredientParts  \
1           39  c("saffron", "milk", "hot green chili peppers"...   
2           40  c("sugar", "lemons, rind of", "lemon, zest of"...   
3           41  c("extra firm tofu", "eggplant", "zucchini", "...   
5           43  c("graham cracker crumbs", "sugar", "butter", ...   
6           44  c("chicken", "butter", "flour", "milk", "celer...   
...        ...                                                ...   
2994      4656  c("dry red wine", "garlic cloves", "Tabasco sa...   
2996      4658    c("butter", "eggs", "flour", "vegetable broth")   
2997      4659  c("bulgur", "honeydew melon", "grapes", "raspb...   
2998      4660  c("turkey", "salt", "paprika", "white pepper",...   
2999      4661  c("yellow corn grits", "chicken broth", "garli...   

                             RecipeIngredientQuantities  Quanti_num  \
1     c("1", "4", "2", "2", "8", "1/4", "8", "1/2", 

In [21]:
test = df[df['Quanti_num'] < df['Ingredient_num']]['RecipeId']
print(test)

2         40
22        60
52        92
66       109
71       115
        ... 
2946    4594
2957    4605
2970    4625
2980    4637
2992    4653
Name: RecipeId, Length: 201, dtype: int64


In [22]:
test = df[df['RecipeId'] == 92]
for key,value in test['RecipeIngredientParts'].items():
    print(value)

for key,value in test['RecipeIngredientQuantities'].items():
    print(value)

c("oranges", "grapefruit juice", "seedless grapes", "honey", "oranges", "grapefruit section", "lemon juice", "lime juice", "walnuts", "apple")
c("1/2", "1", "1", "1", "1", "1/4", "1")


In [23]:
for key,value in value_not_equier.iloc[3][['RecipeIngredientParts']].items():
    print(value)

print(value_not_equier.iloc[3]["RecipeId"])
for key,value in value_not_equier.iloc[3][['RecipeIngredientQuantities']].items():
    print(value)

c("graham cracker crumbs", "sugar", "butter", "sugar", "cornstarch", "salt", "milk", "vanilla extract", "water", "gelatin", "rum", "cream of tartar", "sugar")
43
c("1 1/4", "1/4", "6", "1/3", "1/4", "1/4", "2", "3", "1", "1", "1/4", "1", "2", "3", "1/4", "1/2", NA)


In [26]:
def parse_r_list(r_str):
    """
    将R格式的c("a", "b", NA)字符串解析为Python列表，同时过滤NA和空值
    :param r_str: R格式的字符串（如c("oranges", "honey", NA)）
    :return: 无NA、无空值的Python列表
    """
    if pd.isna(r_str):  # 先判断输入本身是否为NaN
        return []

    # 步骤1：去除开头的c(和结尾的)，统一格式
    cleaned = re.sub(r'^c\(|(?<=\D)\)$', '', r_str)  # 去掉 "c(" 和 非数字结尾的 ")"
    # 步骤2：按 ", " 分割成单个元素（处理如 "a", "b", NA 这样的格式）
    raw_items = cleaned.split(', ')

    # 步骤3：过滤 NA、空字符串，同时去除每个元素的引号
    valid_items = []
    for item in raw_items:
        # 去除元素前后的引号（如 "oranges" → oranges）
        stripped = item.strip().strip('"')
        # 过滤 NA 和空值（注意：R中的NA可能是大写NA，也可能是小写na，统一判断）
        if stripped.upper() != "NA" and stripped != "":
            valid_items.append(stripped)

    return valid_items

df["Ingredient_list"] = df["RecipeIngredientParts"].apply(parse_r_list)
df["Quantity_list"] = df["RecipeIngredientQuantities"].apply(parse_r_list)

test = df[df["RecipeId"] == 43]['Ingredient_list']
for t in test:
    print(t)
print("成功转化为列表")

['graham cracker crumbs', 'sugar', 'butter', 'sugar', 'cornstarch', 'salt', 'milk', 'vanilla extract', 'water', 'gelatin', 'rum', 'cream of tartar', 'sugar']
成功转化为列表


In [25]:
#重新来，看看是否有所改善
df['Quanti_num'] = df['Ingredient_list'].apply(len)
df['Ingredient_num'] = df['Quantity_list'].apply(len)
value_not_equier = df[df['Quanti_num'] != df['Ingredient_num']][
        ["RecipeId","Ingredient_list","Quantity_list","Quanti_num", "Ingredient_num"]
    ]
print("针对食材数量和食材不匹配得到的数据形状")
print(value_not_equier.shape)

print("针对食材数量和食材不匹配的具体情况")
print(value_not_equier)

针对食材数量和食材不匹配得到的数据形状
(2156, 5)
针对食材数量和食材不匹配的具体情况
      RecipeId                                    Ingredient_list  \
2           40  [sugar, lemons, rind of, lemon, zest of, fresh...   
3           41  [extra firm tofu, eggplant, zucchini, mushroom...   
5           43  [graham cracker crumbs, sugar, butter, sugar, ...   
6           44  [chicken, butter, flour, milk, celery, button ...   
7           45  [sugar, margarine, egg, flour, salt, buttermil...   
...        ...                                                ...   
2992      4653  [butter, margarine, brown sugar, all-purpose f...   
2994      4656  [dry red wine, garlic cloves, Tabasco sauce, d...   
2996      4658             [butter, eggs, flour, vegetable broth]   
2997      4659  [bulgur, honeydew melon, grapes, raspberries, ...   
2998      4660  [turkey, salt, paprika, white pepper, cayenne,...   

                                          Quantity_list  Quanti_num  \
2                                [1 1/2, 1, 1 1/2, 3

In [27]:
test = df[df["RecipeId"] == 43]['Ingredient_list']
print('转换前的43号')
for t in test:
    print(t)

转换前的43号
['graham cracker crumbs', 'sugar', 'butter', 'sugar', 'cornstarch', 'salt', 'milk', 'vanilla extract', 'water', 'gelatin', 'rum', 'cream of tartar', 'sugar']


In [38]:

def delSameIng(ings):
    if len(ings) == 0:
        return[]
    d = dict()
    for ing in ings:
        d[ing] = ing
    return list(d.values())

df['Ingredient_list'] = df['Ingredient_list'].apply(delSameIng)
#重新来，看看是否有所改善
df['Ingredient_num'] = df['Ingredient_list'].apply(len)
df['Quanti_num'] = df['Quantity_list'].apply(len)
value_not_equier = df[df['Quanti_num'] != df['Ingredient_num']][
        ["RecipeId","Ingredient_list","Quantity_list","Quanti_num", "Ingredient_num"]
    ]



In [39]:
test = df[df["RecipeId"] == 43]['Ingredient_list']
print('转换后的43号')
for t in test:
    print(t)
print("去重成功")

转换后的43号
['graham cracker crumbs', 'sugar', 'butter', 'cornstarch', 'salt', 'milk', 'vanilla extract', 'water', 'gelatin', 'rum', 'cream of tartar']
去重成功


In [45]:
#切割食材大于食量的函数
def split_len(longer, smaller):
    return longer[:len(smaller)]

mask = df.apply(lambda row: len(row['Ingredient_list'])>len(row['Quantity_list']), axis=1)

df['Ingredient_num'] = df['Ingredient_list'].apply(len)
df['Quanti_num'] = df['Quantity_list'].apply(len)

print(df.loc[mask, ['RecipeId','Ingredient_num','Quanti_num']])
print("未改变前的第40号元素")
test = df[df['RecipeId'] == 40]['Ingredient_list']
test2 = df[df['RecipeId'] == 40]['Quantity_list']

for v in test:
    print(v)

for v in test2:
    print(v)

      RecipeId  Ingredient_num  Quanti_num
2           40               7           4
6           44              12          11
13          51              11          10
21          59               9           5
37          76               5           4
...        ...             ...         ...
2970      4625               7           6
2972      4627              16          15
2977      4633              15          14
2992      4653              13          11
2996      4658               4           3

[489 rows x 3 columns]
未改变前的第40号元素
['sugar', 'lemons', 'rind of', 'lemon', 'zest of', 'fresh water', 'fresh lemon juice']
['1 1/2', '1', '1 1/2', '3/4']


In [49]:
df.loc[mask,'Ingredient_list']= df.apply(lambda row: split_len(
    row['Ingredient_list'],
    row['Quantity_list']
), axis=1)

df['Ingredient_num'] = df['Ingredient_list'].apply(len)
df['Quanti_num'] = df['Quantity_list'].apply(len)

print("改变后的第40号元素")
test = df[df['RecipeId'] == 40]['Ingredient_list']
test2 = df[df['RecipeId'] == 40]['Quantity_list']

for v in test:
    print(v)

for v in test2:
    print(v)

改变后的第40号元素
['sugar', 'lemons', 'rind of', 'lemon']
['1 1/2', '1', '1 1/2', '3/4']


In [51]:
print("再看看食材量大于食材数的")
print(df[df['Ingredient_num']!=df['Quanti_num']][['RecipeId','Ingredient_num','Quanti_num']])

再看看食材量大于食材数的
      RecipeId  Ingredient_num  Quanti_num
3           41              14          15
5           43              11          16
7           45               7          10
8           46               2           6
9           47               6           7
...        ...             ...         ...
2990      4649               2           4
2991      4650               8           9
2994      4656               7           8
2997      4659               5           7
2998      4660               6           7

[1733 rows x 3 columns]


In [57]:
print("未改变的第43号元素")
test = df[df['RecipeId'] == 43]['Ingredient_list']
test2 = df[df['RecipeId'] == 43]['Quantity_list']

for v in test:
    print(v)

for v in test2:
    print(v)

mask = df['Ingredient_num']<df['Quanti_num']
df.loc[mask,'Quantity_list']= df.apply(lambda row: split_len(
    row['Quantity_list'],
    row['Ingredient_list']
),axis=1)

df['Ingredient_num'] = df['Ingredient_list'].apply(len)
df['Quanti_num'] = df['Quantity_list'].apply(len)

print("改变的第43号元素")
test = df[df['RecipeId'] == 43]['Ingredient_list']
test2 = df[df['RecipeId'] == 43]['Quantity_list']

for v in test:
    print(v)

for v in test2:
    print(v)

未改变的第43号元素
['graham cracker crumbs', 'sugar', 'butter', 'cornstarch', 'salt', 'milk', 'vanilla extract', 'water', 'gelatin', 'rum', 'cream of tartar']
['1 1/4', '1/4', '6', '1/3', '1/4', '1/4', '2', '3', '1', '1', '1/4', '1', '2', '3', '1/4', '1/2']
改变的第43号元素
['graham cracker crumbs', 'sugar', 'butter', 'cornstarch', 'salt', 'milk', 'vanilla extract', 'water', 'gelatin', 'rum', 'cream of tartar']
['1 1/4', '1/4', '6', '1/3', '1/4', '1/4', '2', '3', '1', '1', '1/4']


In [60]:
print(df[df['Ingredient_num'] != df['Quanti_num']][['RecipeId','Ingredient_num','Quanti_num']])
print(len(df['Ingredient_list'])!=len(df['Quantity_list']))

Empty DataFrame
Columns: [RecipeId, Ingredient_num, Quanti_num]
Index: []
False


In [61]:
import time
for i in range(3):
    print(f"批次{i+1}完成")
    time.sleep(2)  # 暂停2秒模拟处理时间

批次1完成
批次2完成
批次3完成
